In [34]:
#coding=utf-8
!python --version
!pwd
!ls

Python 3.7.5
/d/GH/GitWorkSpace/bank_model_competiton
0825_lgb_base_cv_v0.1.csv
0825_lgb_base_cv_v0.2.csv
0825_lgb_base_cv_v0.3.csv
data_eda.py
model.ipynb
process.ipynb
process.py
test.dat
testaa
testaa.csv
testaa_bank_statement.csv
testaa_submit_example.csv
train
train.csv
train.dat
train_bank_statement.csv
~$姣旇禌瀛楁�佃В閲�.xlsx
鍒濊禌A姒滄暟鎹�闆�
鏁版嵁鎺㈡煡.ipynb
姣旇禌瀛楁�佃В閲�.xlsx


In [20]:
# -*- coding: utf-8 -*-
from heamy.dataset import Dataset
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
import pandas as pd
import xgboost as xgb
import datetime
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
import sys

dummy_fea_str = "id,title,career,zip_code,residence,syndicated,installment,level"
dummy_fea = dummy_fea_str.split(',')

In [31]:

def xgb_feature(X_train, y_train, X_test, y_test=None):
    # 模型参数
    params = {'booster': 'gbtree',
              'objective':'rank:pairwise',
              'eval_metric' : 'auc',
              'eta': 0.02,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 1,  # 2 3
              'seed': 1111,
              'silent':1
              }
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvali = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, num_boost_round=800)
    predict = model.predict(dvali)
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)

def xgb_feature2(X_train, y_train, X_test, y_test=None):
    # 模型参数
    params = {'booster': 'gbtree',
              'objective':'rank:pairwise',
              'eval_metric' : 'auc',
              'eta': 0.015,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 1,  # 2 3
              'seed': 11,
              'silent':1
              }
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvali = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, num_boost_round=1200)
    predict = model.predict(dvali)
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)

def xgb_feature3(X_train, y_train, X_test, y_test=None):
    # 模型参数
    params = {'booster': 'gbtree',
              'objective':'rank:pairwise',
              'eval_metric' : 'auc',
              'eta': 0.01,
              'max_depth': 5,  # 4 3
              'colsample_bytree': 0.7,#0.8
              'subsample': 0.7,
              'min_child_weight': 1,  # 2 3
              'seed': 1,
              'silent':1
              }
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvali = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, num_boost_round=2000)
    predict = model.predict(dvali)
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)


def et_model(X_train, y_train, X_test, y_test=None):
    model = ExtraTreesClassifier(max_features = 'log2', n_estimators = 1000 , n_jobs = -1).fit(X_train,y_train)
    return model.predict_proba(X_test)[:,1]

def gbdt_model(X_train, y_train, X_test, y_test=None):
    model = GradientBoostingClassifier(learning_rate = 0.02, max_features = 0.7, n_estimators = 700 , max_depth = 5).fit(X_train,y_train)
    predict = model.predict_proba(X_test)[:,1]
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)

def logistic_model(X_train, y_train, X_test, y_test=None):
    model = LogisticRegression(penalty = 'l2').fit(X_train,y_train)
    return model.predict_proba(X_test)[:,1]

def lgb_feature(X_train, y_train, X_test, y_test=None):
    lgb_train = lgb.Dataset(X_train, y_train,categorical_feature={'sex', 'merriage', 'income', 'qq_bound', 'degree', 'wechat_bound','account_grade','industry'})
    lgb_test = lgb.Dataset(X_test,categorical_feature={'sex', 'merriage', 'income', 'qq_bound', 'degree', 'wechat_bound','account_grade','industry'})
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric':'auc',
        'num_leaves': 25,
        'learning_rate': 0.01,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'min_data_in_leaf':5,
        'max_bin':200,
        'verbose': 0,
    }
    gbm = lgb.train(params,lgb_train,num_boost_round=2000)
    predict = gbm.predict(X_test)
    minmin = min(predict)
    maxmax = max(predict)
    vfunc = np.vectorize(lambda x:(x-minmin)/(maxmax-minmin))
    return vfunc(predict)


## 验证v1

In [33]:
from sklearn.model_selection import train_test_split

dummy_fea_str = "id,title,career,zip_code,residence,syndicated,installment,level"
dummy_fea = dummy_fea_str.split(',')

sample_data = pd.read_csv('train.dat', engine = 'python');
sample_data = sample_data.fillna(0)
dummy_df = pd.get_dummies(sample_data.loc[:,dummy_fea])
# print(dummy_df)

sample_data_concat = pd.concat([sample_data, dummy_df], axis=1)
sample_data_concat =sample_data_concat.fillna(0)
vaild_sample_data = sample_data_concat.drop(dummy_fea, axis=1)
sample_data_target = vaild_sample_data['label'].values
print(vaild_sample_data.columns)
sample_data_x = vaild_sample_data.drop(['label'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(sample_data_x, sample_data_target, test_size=0.2, random_state=None, shuffle=True)

# logistic model
predict_result = logistic_model(X_train, y_train, X_test, None)
print('logistic_model valid auc', roc_auc_score(y_test, predict_result))

# gbdt model
predict_result = gbdt_model(X_train, y_train, X_test, None)
print('gbdt_model valid auc', roc_auc_score(y_test, predict_result))



Index(['Unnamed: 0', 'loan', 'term', 'interest_rate', 'issue_time',
       'record_time', 'history_time', 'total_accounts', 'balance_accounts',
       'balance_limit', 'balance', 'label', '0', '1', '2', '3', '4', '5', '6',
       '7', 'level_A0', 'level_A1', 'level_A2', 'level_A3', 'level_A4',
       'level_A5', 'level_B0', 'level_B1', 'level_B2', 'level_B3', 'level_B4',
       'level_B5', 'level_C1', 'level_C2', 'level_C3', 'level_C4', 'level_C5',
       'level_D1', 'level_D2', 'level_D3', 'level_D4', 'level_D5', 'level_E1',
       'level_E2', 'level_E3', 'level_E4', 'level_E5'],
      dtype='object')
valid auc 0.5244078016628604
valid auc 0.6572872490143379


## 验证v2

In [36]:
from sklearn.model_selection import train_test_split

dummy_fea_str = "id,title,career,zip_code,residence,syndicated,installment,level"
dummy_fea = dummy_fea_str.split(',')

sample_data = pd.read_csv('train.dat', engine = 'python');
sample_data = sample_data.fillna(0)
dummy_df = pd.get_dummies(sample_data.loc[:,dummy_fea])
print('dummy_df \n',dummy_df)

sample_data_concat = pd.concat([sample_data, dummy_df], axis=1)
sample_data_concat =sample_data_concat.fillna(0)
vaild_sample_data = sample_data_concat.drop(dummy_fea, axis=1)
sample_data_target = vaild_sample_data['label'].values
print(vaild_sample_data.columns)
sample_data_x = vaild_sample_data.drop(['label'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(sample_data_x, sample_data_target, test_size=0.2, random_state=None, shuffle=True)

xgb_dataset = Dataset(X_train=X_train, y_train=y_train, X_test = X_test, y_test= None, use_cache=False)

model_xgb  = Regressor(dataset=xgb_dataset, estimator=xgb_feature,name='xgb',use_cache=False)
model_xgb2 = Regressor(dataset=xgb_dataset, estimator=xgb_feature2,name='xgb2',use_cache=False)
model_xgb3 = Regressor(dataset=xgb_dataset, estimator=xgb_feature3,name='xgb3',use_cache=False)
model_gbdt = Regressor(dataset=xgb_dataset, estimator=gbdt_model,name='gbdt',use_cache=False)

pipeline = ModelsPipeline(model_xgb, model_xgb2, model_xgb3, model_gbdt)
stack_ds = pipeline.stack(k=4, seed=111, add_diff=False, full_test=True)
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression, parameters={'fit_intercept': False})
predict_result = stacker.predict()

print('ModelsPipeline(model_xgb, model_xgb2, model_xgb3, model_gbdt) valid auc', roc_auc_score(y_test, predict_result))


Index(['Unnamed: 0', 'loan', 'term', 'interest_rate', 'issue_time',
       'record_time', 'history_time', 'total_accounts', 'balance_accounts',
       'balance_limit', 'balance', 'label', '0', '1', '2', '3', '4', '5', '6',
       '7', 'level_A0', 'level_A1', 'level_A2', 'level_A3', 'level_A4',
       'level_A5', 'level_B0', 'level_B1', 'level_B2', 'level_B3', 'level_B4',
       'level_B5', 'level_C1', 'level_C2', 'level_C3', 'level_C4', 'level_C5',
       'level_D1', 'level_D2', 'level_D3', 'level_D4', 'level_D5', 'level_E1',
       'level_E2', 'level_E3', 'level_E4', 'level_E5'],
      dtype='object')
[19:58:46] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if yo

## lgb feature

In [37]:
train_data = pd.read_csv('train.dat',engine = 'python');
train_data = train_data.fillna(0)

test_data = pd.read_csv('test.dat',engine = 'python');
test_data = test_data.fillna(0)

train_test_data = pd.concat([train_data, test_data],axis=0,ignore_index = True)
train_test_data = train_test_data.fillna(0)

train_data = train_test_data.iloc[:train_data.shape[0],:]
test_data = train_test_data.iloc[train_data.shape[0]:,:]

#处理dummy变量
for _fea in dummy_fea:
    print(_fea)
    le = LabelEncoder()
    le.fit(train_data[_fea].tolist() + test_data[_fea].tolist())
    tmp = le.transform(train_data[_fea].tolist() + test_data[_fea].tolist())
    train_data[_fea] = tmp[:train_data.shape[0]]
    test_data[_fea]  = tmp[train_data.shape[0]:]
    
train_x = train_data.drop(['label'],axis=1)
test_x = test_data.drop(['label'],axis=1)
lgb_dataset = Dataset(train_x, train_data['label'], test_x, use_cache=False)

id
title
career
zip_code
residence
syndicated
installment
level


d:\users\chenchen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\users\chenchen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## xgb feature

In [ ]:
train_data = pd.read_csv('train.dat',engine = 'python');
train_data = train_data.fillna(0)

test_data = pd.read_csv('test.dat',engine = 'python');
test_data = test_data.fillna(0)

train_test_data = pd.concat([train_data, test_data], axis=0, ignore_index = True)
train_test_data = train_test_data.fillna(0)

dummy_df = pd.get_dummies(train_test_data.loc[:,dummy_fea])
train_test_data = pd.concat([train_test_data, dummy_df],axis=1)
train_test_data = train_test_data.drop(dummy_fea, axis=1)

train_train = train_test_data.iloc[:train_data.shape[0],:]
test_test = train_test_data.iloc[train_data.shape[0]:,:]

train_train_x = train_train.drop(['target'],axis=1)
test_test_x = test_test.drop(['target'],axis=1)

xgb_dataset = Dataset(X_train=train_train_x, y_train=train_train['target'], X_test = test_test_x, y_test= None, use_cache=False)

## stacking

In [ ]:
model_xgb = Regressor(dataset=xgb_dataset, estimator=xgb_feature,name='xgb',use_cache=False)
model_xgb2 = Regressor(dataset=xgb_dataset, estimator=xgb_feature2,name='xgb2',use_cache=False)
model_xgb3 = Regressor(dataset=xgb_dataset, estimator=xgb_feature3,name='xgb3',use_cache=False)
model_lgb = Regressor(dataset=lgb_dataset, estimator=lgb_feature,name='lgb',use_cache=False)
model_gbdt = Regressor(dataset=xgb_dataset, estimator=gbdt_model,name='gbdt',use_cache=False)
pipeline = ModelsPipeline(model_xgb, model_xgb2, model_xgb3, model_gbdt)
stack_ds = pipeline.stack(k=5, seed=111, add_diff=False, full_test=True)
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression,parameters={'fit_intercept': False})
predict_result = stacker.predict()

In [ ]:
ans = pd.read_csv('testaa.csv')
ans['PROB'] = predict_result
minmin = min(ans['PROB']),
maxmax = max(ans['PROB'])
ans['PROB'] = ans['PROB'].map(lambda x:(x-minmin)/(maxmax-minmin))
ans['PROB'] = ans['PROB'].map(lambda x:'%.4f' % x)
ans.to_csv('./ans_stacking.csv',index=None)